In [19]:
from tqdm import tqdm
import cv2

from dataset.sport_smot_dataset import SportSmotDataset
from dataset.sequence import Sequence

dataset = SportSmotDataset('/home/rafa/Downloads/sportsmot_publish/dataset')
len(dataset.raw_videos)


90

In [20]:
from models.i3d import I3D

i3d = I3D(feat_ext=True)
det_w = det_h = 244


In [21]:
from copy import copy
import pickle as pkl


def remove_ellipsis(sequence):
    first_non_ellipsis = None
    for idx, el in enumerate(sequence):
        if el is not ...:
            first_non_ellipsis = el
            break
    if first_non_ellipsis is None:
        return sequence

    current_non_ellipsis = first_non_ellipsis
    for idx, el in enumerate(sequence):
        if el is not ...:
            current_non_ellipsis = el
        else:
            sequence[idx] = copy(current_non_ellipsis)
    return sequence


sequences = []
for raw_video in tqdm(dataset.raw_videos):
    gt = raw_video.load_pd()
    vid_len = gt['frame'].max()
    chunks = [(i, i + 31) for i in range(1, vid_len + 1, 32)]
    if chunks[-1][1] > vid_len:
        del chunks[-1]

    for chunk in chunks:
        is_in_frames = ((gt['frame'] >= chunk[0]) & (gt['frame'] <= chunk[1]))
        person_ids = gt[is_in_frames]['person_id'].unique()
        person_to_frames = {}
        for person_id in person_ids:
            person_frames = []
            max_w = gt[is_in_frames & (gt['person_id'] == person_id)]['w'].max()
            max_h = gt[is_in_frames & (gt['person_id'] == person_id)]['h'].max()
            for frame in range(chunk[0], chunk[1] + 1):
                label = gt[(gt['frame'] == frame) & (gt['person_id'] == person_id)].to_dict('r')
                if len(label) > 1:
                    raise ValueError('person id not unique for one frame')
                if len(label) == 0:
                    person_frames.append(...)
                    continue
                label = label[0]

                img = cv2.imread(raw_video.name_pattern.format(frame=frame))
                roi = img[label['y1']: label['y1'] + max_h, label['x1']: label['x1'] + max_w]
                roi = cv2.resize(roi, (det_w, det_h), interpolation=cv2.INTER_AREA)
                person_frames.append(roi)
            cleared_frames = remove_ellipsis(person_frames)
            if cleared_frames:
                person_to_frames[person_id] = i3d.inference(cleared_frames)
        sequences.append(Sequence(
            gt=gt[is_in_frames],
            name_pattern=raw_video.name_pattern,
            klass=raw_video.klass,
            person_features=person_to_frames
        ))
        with open("sequences.pkl", 'wb') as file:
            pkl.dump(sequences, file)


100%|██████████| 90/90 [2:05:39<00:00, 83.77s/it]   


In [24]:
import pickle as pkl
from random import shuffle

shuffle(sequences)
with open("sequences.pkl", 'wb') as file:
    pkl.dump(sequences, file)